In [ ]:
import os
import warnings
from dotenv import load_dotenv

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings("ignore")

load_dotenv()

In [ ]:
from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS 
from langchain_community.docstore.in_memory import InMemoryDocstore

In [ ]:
embeddings = OllamaEmbeddings(model='data', base_url='http://localhost:11434')

db_name = r"data"
vector_store = FAISS.load_local(db_name, embeddings, allow_dangerous_deserialization=True)

In [ ]:
question = "how to gain muscle mass?"
docs = vector_store.search(query=question, k=5, search_type="similarity")

In [ ]:
docs

In [ ]:
retriever = vector_store.as_retriever(search_type = 'similarity', 
                                      search_kwargs = {'k': 3})

retriever.invoke(question)

In [ ]:
question = "how to lose weight?"
retriever.invoke(question)

In [ ]:
retriever = vector_store.as_retriever(search_type = 'similarity_score_threshold', 
                                      search_kwargs = {'k': 3, 'score_threshold': 0.1})

retriever.invoke(question)

In [ ]:
retriever = vector_store.as_retriever(search_type = 'mmr', 
                                      search_kwargs = {'k': 3, 'fetch_k': 20, 'lambda_mult': 1})

docs = retriever.invoke(question)
docs

In [ ]:
from langchain_ollama import ChatOllama 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.prompts import ChatPromptTemplate

from langchain import hub

In [ ]:
prompt = hub.pull("data")
prompt

In [ ]:
# prompt = """
# You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
# If you don't know the answer, just say that you don't know.
# Use three sentences maximum and keep the answer concise.

# Question: {question} 
# Context: {context} 
# Answer:"""


prompt = """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
    Question: {question} 
    Context: {context} 
    Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt)
# prompt

In [ ]:
llm = ChatOllama(model='data', base_url='http://localhost:11434')
llm.invoke('hi')

In [ ]:
def format_docs(docs):
    return '\n\n'.join([doc.page_content for doc in docs])

context = format_docs(docs)
# print(context)

In [ ]:
rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question = "how to lose weight?"
response = rag_chain.invoke(question)

print(response)

In [ ]:
question = "how to gain muscle mass?"
response = rag_chain.invoke(question)

print(response)